In [1]:
import pandas as pd
import duckdb
import seaborn as sns
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
#duckdb.sql("INSTALL postgres;")
duckdb.sql("load postgres;")

In [4]:
# attach postgres database
duckdb.sql(f"ATTACH 'dbname={os.getenv("DB_NAME")} user={os.getenv("DB_USER")} host={os.getenv("DB_HOST")} password={os.getenv("DB_PASSWORD")} port={os.getenv("DB_PORT")}' AS supabase (TYPE postgres, SCHEMA 'public');")

In [5]:
# initial query to test
duckdb.sql("select * from supabase.anime limit 10;").df()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [6]:
genre_denest_query="""
select 
  lower(trim(UNNEST(STRING_SPLIT(genre, ','))))
from supabase.anime
group by genre
"""

duckdb.sql(genre_denest_query)

┌──────────────────────────────────────────────────────┐
│ lower(main."trim"(unnest(string_split(genre, ',')))) │
│                       varchar                        │
├──────────────────────────────────────────────────────┤
│ action                                               │
│ adventure                                            │
│ fantasy                                              │
│ mystery                                              │
│ police                                               │
│ psychological                                        │
│ supernatural                                         │
│ thriller                                             │
│ action                                               │
│ drama                                                │
│   ·                                                  │
│   ·                                                  │
│   ·                                                  │
│ mystery                      

In [7]:
genre_query="""
select 
  lower(trim(UNNEST(STRING_SPLIT(genre, ',')))) AS genre,
  count(genre) as genre_popularity
from supabase.anime
group by genre
order by genre_popularity desc;
"""
duckdb.sql(genre_query)

┌───────────────┬──────────────────┐
│     genre     │ genre_popularity │
│    varchar    │      int64       │
├───────────────┼──────────────────┤
│ comedy        │              245 │
│ music         │              196 │
│ slice of life │               93 │
│ comedy        │               93 │
│ fantasy       │               63 │
│ sci-fi        │               62 │
│ mecha         │               62 │
│ action        │               62 │
│ slice of life │               60 │
│ school        │               60 │
│   ·           │                · │
│   ·           │                · │
│   ·           │                · │
│ martial arts  │                1 │
│ seinen        │                1 │
│ sports        │                1 │
│ action        │                1 │
│ adventure     │                1 │
│ fantasy       │                1 │
│ kids          │                1 │
│ magic         │                1 │
│ shoujo        │                1 │
│ super power   │                1 │
├

In [8]:
type_query ="""
SELECT
  type,
  count(type) as type_popularity
from supabase.anime
group by type
order by type_popularity desc
"""

duckdb.sql(type_query)

┌─────────┬─────────────────┐
│  type   │ type_popularity │
│ varchar │      int64      │
├─────────┼─────────────────┤
│ TV      │            2939 │
│ OVA     │            1438 │
│ Movie   │            1296 │
│ Special │            1294 │
│ ONA     │             324 │
│ Music   │             295 │
└─────────┴─────────────────┘

In [9]:
type_rating_query ="""
SELECT
  type,
  avg(rating)::decimal(15,2) as average_rating,
  median(rating) as median_rating,
  min(rating) as min_rating,
  max(rating) as max_rating,
from supabase.anime
group by type
order by average_rating desc, median_rating desc 
"""

duckdb.sql(type_rating_query)

┌─────────┬────────────────┬───────────────┬────────────┬────────────┐
│  type   │ average_rating │ median_rating │ min_rating │ max_rating │
│ varchar │ decimal(15,2)  │    double     │   double   │   double   │
├─────────┼────────────────┼───────────────┼────────────┼────────────┤
│ TV      │           7.12 │          7.11 │       5.28 │       9.26 │
│ Movie   │           6.98 │          7.02 │       5.27 │       9.37 │
│ Special │           6.79 │          6.78 │       5.27 │       8.66 │
│ OVA     │           6.73 │          6.68 │       5.27 │       9.11 │
│ ONA     │           6.34 │          6.24 │       5.27 │       8.26 │
│ Music   │           6.16 │          6.06 │       5.27 │       8.38 │
└─────────┴────────────────┴───────────────┴────────────┴────────────┘

In [10]:
type_members_query ="""
SELECT
  type,
  avg(members)::int64() as average_members,
  median(members)::int64() as median_members,
  min(members) as min_members,
  max(members) as max_members,
from supabase.anime
group by type
order by average_members desc, median_members desc 
"""

duckdb.sql(type_members_query)

┌─────────┬─────────────────┬────────────────┬─────────────┬─────────────┐
│  type   │ average_members │ median_members │ min_members │ max_members │
│ varchar │      int64      │     int64      │    int64    │    int64    │
├─────────┼─────────────────┼────────────────┼─────────────┼─────────────┤
│ TV      │           54572 │          14718 │         131 │     1013917 │
│ Movie   │           18231 │           2721 │         104 │      466254 │
│ OVA     │           11212 │           3620 │         136 │      305165 │
│ Special │            9838 │           3628 │         110 │      160423 │
│ ONA     │            8041 │           1466 │          97 │      144898 │
│ Music   │            2056 │            522 │          82 │       71136 │
└─────────┴─────────────────┴────────────────┴─────────────┴─────────────┘

In [11]:
episodes_query="""
select distinct episodes
from supabase.anime
"""

duckdb.sql(episodes_query)

┌────────────┐
│  episodes  │
│  varchar   │
├────────────┤
│ 64         │
│ 148        │
│ 110        │
│ 13         │
│ 26         │
│ 47         │
│ 23         │
│ 40         │
│ 38         │
│ 175        │
│  ·         │
│  ·         │
│  ·         │
│ 167        │
│ 73         │
│ 29         │
│ 115        │
│ 193        │
│ 66         │
│ 296        │
│ 140        │
│ 526        │
│ 237        │
├────────────┤
│  164 rows  │
│ (20 shown) │
└────────────┘

In [12]:
episodes_query="""
select cast(episodes as INTEGER)
from supabase.anime
"""

duckdb.sql(episodes_query)

ConversionException: Conversion Error: Could not convert string 'Unknown' to INT32 when casting from source column episodes

LINE 1: SELECT CAST(episodes AS INTEGER) FROM supabase.anime LIMIT 10000
                ^

In [13]:
rating_table_query = """
select * from supabase.rating
limit 10;
"""

duckdb.sql(rating_table_query)

┌─────────┬──────────┬────────┐
│ user_id │ anime_id │ rating │
│  int64  │  int64   │ int64  │
├─────────┼──────────┼────────┤
│       1 │       20 │     -1 │
│       1 │       24 │     -1 │
│       1 │       79 │     -1 │
│       1 │      226 │     -1 │
│       1 │      241 │     -1 │
│       1 │      355 │     -1 │
│       1 │      356 │     -1 │
│       1 │      442 │     -1 │
│       1 │      487 │     -1 │
│       1 │      846 │     -1 │
├─────────┴──────────┴────────┤
│ 10 rows           3 columns │
└─────────────────────────────┘

In [14]:
rating_user_query = """
select
  user_id,
  count(user_id) as user_activity,
  avg(rating)::int64() as average_rating,
  median(rating)::int64() as median_rating,
  max(rating) as max_rating,
  min(rating) as min_rating
from supabase.rating
group by user_id
order by user_activity desc
"""

duckdb.sql(rating_user_query)

┌─────────┬───────────────┬────────────────┬───────────────┬────────────┬────────────┐
│ user_id │ user_activity │ average_rating │ median_rating │ max_rating │ min_rating │
│  int64  │     int64     │     int64      │     int64     │   int64    │   int64    │
├─────────┼───────────────┼────────────────┼───────────────┼────────────┼────────────┤
│   48766 │         10227 │             -1 │            -1 │         -1 │         -1 │
│   42635 │          3747 │              6 │             6 │         10 │          5 │
│   53698 │          2905 │              7 │             7 │         10 │          1 │
│   57620 │          2702 │              8 │             8 │         10 │         -1 │
│   59643 │          2633 │              7 │             7 │         10 │         -1 │
│   51693 │          2622 │              7 │             7 │         10 │         -1 │
│   27364 │          2499 │              3 │            -1 │         10 │         -1 │
│   45659 │          2469 │              7 

In [19]:
anime_user_rating_query = """
select
  a.name,
  a.anime_id,
  avg(r.rating)::int64() as average_rating,
  median(r.rating)::int64() as median_rating,
from supabase.anime as a, supabase.rating as r
where a.anime_id = r.anime_id
group by a.name, a.anime_id
order by average_rating desc, median_rating desc
"""

duckdb.sql(anime_user_rating_query)

┌───────────────────────────────────────────────────────┬──────────┬────────────────┬───────────────┐
│                         name                          │ anime_id │ average_rating │ median_rating │
│                        varchar                        │  int64   │     int64      │     int64     │
├───────────────────────────────────────────────────────┼──────────┼────────────────┼───────────────┤
│ STAR BEAT!: Hoshi no Kodou                            │    34252 │             10 │            10 │
│ Tang Lang Bu Chan                                     │    31972 │              9 │             9 │
│ Gintama°                                              │    28977 │              8 │            10 │
│ Ginga Eiyuu Densetsu                                  │      820 │              8 │            10 │
│ Kimi no Na wa.                                        │    32281 │              8 │            10 │
│ Cross Game                                            │     5941 │              

In [22]:
anime_user_rating_type_query = """
select
  a.type,
  avg(r.rating)::int64() as average_rating,
  median(r.rating)::int64() as median_rating,
from supabase.anime as a, supabase.rating as r
where a.anime_id = r.anime_id
group by a.type
order by average_rating desc, median_rating desc
"""

duckdb.sql(anime_user_rating_type_query)

┌─────────┬────────────────┬───────────────┐
│  type   │ average_rating │ median_rating │
│ varchar │     int64      │     int64     │
├─────────┼────────────────┼───────────────┤
│ Movie   │              6 │             8 │
│ TV      │              6 │             8 │
│ OVA     │              6 │             7 │
│ ONA     │              6 │             7 │
│ Music   │              5 │             7 │
│ Special │              5 │             7 │
└─────────┴────────────────┴───────────────┘

In [31]:
genre_cleaning_1_query="""
select
  lower(trim(UNNEST(STRING_SPLIT(genre, ',')))) AS genre,
from supabase.anime 
"""

duckdb.sql(genre_cleaning_1_query)

┌────────────────────────┐
│         genre          │
│        varchar         │
├────────────────────────┤
│ drama                  │
│ romance                │
│ school                 │
│ supernatural           │
│ action                 │
│ adventure              │
│ drama                  │
│ fantasy                │
│ magic                  │
│ military               │
│   ·                    │
│   ·                    │
│   ·                    │
│ mecha                  │
│ military               │
│ romance                │
│ sci-fi                 │
│ shounen                │
│ action                 │
│ mecha                  │
│ military               │
│ romance                │
│ shoujo                 │
├────────────────────────┤
│         ? rows         │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [32]:
genre_cleaning_2_query="""
select
  distinct lower(trim(UNNEST(STRING_SPLIT(genre, ',')))) AS genre,
from supabase.anime 
"""

duckdb.sql(genre_cleaning_2_query)

┌───────────────┐
│     genre     │
│    varchar    │
├───────────────┤
│ magic         │
│ sci-fi        │
│ super power   │
│ game          │
│ shounen       │
│ thriller      │
│ shounen ai    │
│ adventure     │
│ shoujo        │
│ cars          │
│  ·            │
│  ·            │
│  ·            │
│ sports        │
│ slice of life │
│ mecha         │
│ mystery       │
│ horror        │
│ dementia      │
│ space         │
│ music         │
│ demons        │
│ ecchi         │
├───────────────┤
│    41 rows    │
│  (20 shown)   │
└───────────────┘